## Elliptic Curve Digital Signature Algorithm (ECDSA) in Python From Scratch


### Reference
- [Elliptic Curve Digital Signature Algorithm (ECDSA) Explained](https://www.youtube.com/watch?v=Br8o9KnyPJI&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=12)
- [Elliptic Curve Digital Signature Algorithm (ECDSA) in Python From Scratch](https://www.youtube.com/watch?v=2n1Z6jmzvxs&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=12)
- [Attacking ECDSA with Not Random Keys: The Sony PlayStation 3 Hack](https://www.youtube.com/watch?v=OWv9c0mlppI&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=13)

In [9]:
import random
import hashlib

In [ ]:
# y^2 = x^3 + a* x + b

In [2]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
     
    is_on_curve((x3, y3), p)
         
    return x3, y3
 
def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
     
def apply_double_and_add_method(G, k, p):
    target_point = G
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
     
    is_on_curve(target_point, p)
     
    return target_point

In [3]:
# from :https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)


#finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [4]:
point_2G = add_points(G,G,p)

In [5]:
point_2G

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

In [7]:
tmp_point = G
for i in range(2,21):
    tmp_point = add_points(tmp_point, G, p)
    print(f"{i}G = {tmp_point}")

2G = (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
3G = (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
4G = (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018)
5G = (21505829891763648114329055987619236494102133314575206970830385799158076338148, 98003708678762621233683240503080860129026887322874138805529884920309963580118)
6G = (115780575977492633039504758427830329241728645270042306223540962614150928364886, 78735063515800386211891312544505775871260717697865196436804966483607426560663)
7G = (41948375291644419605210209193538855353224492619856392092318293986323063962044, 48361766907851246668144012348516735800090617714386977531302791340517493990618)
8G = (2126205

In [8]:
apply_double_and_add_method(G= G, k = 20, p= p)

(34773495056115281091786765947597603724784643419904767525769502836017890139287,
 8470533044743364938367028725608288731153024648869546164814808839694950063162)

In [ ]:
# Q  = k x G

## Alice generates her private and public key pair

In [10]:
import random
import hashlib

In [11]:
# private key of Alice
d = random.getrandbits(256)

# public key of Alice
Q = apply_double_and_add_method(G = G, k =d , p=p)


In [42]:
d

108395789411961615835399851759863454075012628118857096644424533495328272294468

In [12]:
random_key = random.getrandbits(256)
random_point = apply_double_and_add_method(G = G, k= random_key , p=p)


In [28]:
random_key

34980700951895659285136384642521748599333592514493469351601238298407345280273

In [29]:
# message = b"ECC beats RSA"
message =  b"ECC beats Diffie-Hellman, too!"
hash_hex= hashlib.sha1(message).hexdigest()
hash_int =int(hash_hex, 16)

In [30]:
hash_int

531204124611006148816545464087066627965678629689

## Sign

In [31]:
# (r,s)
r = ( random_point[0]) % n
s = ( (hash_int + r* d ) * pow(random_key, -1, n)) % n


# s1 = (h1 + r * d) * random_key ^ -1
# s2 = (h2 + r * d) * random_key ^ -1
# s1 - s2 = (h1 + r * d) * random_key ^ -1 - ( h2 + r * d) * random_key ^ -1
# s1 - s2 = random_key & -1 * (h1 + r * d - h2 - r * d)
# s1 - s2 = random_key ^ -1 * (h1- h2)
# (s1 - s2) / (h1 - h2) = random_key ^ -1
# (h1 - h2) / (s1 - s2) = random_key

In [32]:
r

20428615131696736526988868550202295935367423786502498705763296778663460029248

In [33]:
s

4494460473035111833568019675780148601125534298477543159149381137274376904072

## Attacking

- [Attacking ECDSA with Not Random Keys: The Sony PlayStation 3 Hack](https://www.youtube.com/watch?v=OWv9c0mlppI&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=13)

In [35]:
s1 =100991366230175073577062156396065625457283721916183640804389675265266059552088
h1 =320026739459778556085970613903841025917693204146

s2 = 4494460473035111833568019675780148601125534298477543159149381137274376904072
h2 = 531204124611006148816545464087066627965678629689

In [36]:
random_key_prime = (((h1- h2)% n) * pow( s1 - s2, -1, n)) % n

In [37]:
random_key_prime

34980700951895659285136384642521748599333592514493469351601238298407345280273

In [38]:
assert random_key == random_key_prime

In [40]:
random_point_prime = apply_double_and_add_method(G= G, k= random_key_prime, p= p)
r_prime = random_point_prime[0]

In [41]:
assert r_prime ==r

In [ ]:
# s1 = (h1 + r*d) * random_key ^ -1
# s1 * random_key = h1 + r* d
# s1 * random_key -h1 = r * d
# (s1 * random_key - h1 ) / r = d

In [43]:
# 108395789411961615835399851759863454075012628118857096644424533495328272294468
d_prime = ( (s1 * random_key_prime - h1) * pow(r, -1, n)) % n

In [44]:
d_prime

108395789411961615835399851759863454075012628118857096644424533495328272294468

In [45]:
assert d_prime ==d

## Verification

In [23]:
w = pow(s, -1, n)
u1 = apply_double_and_add_method(G = G, k = ( hash_int * w ) % n, p=p)
u2 = apply_double_and_add_method(G = G, k = (r * w) % n, p=p) 

In [24]:
# u1 + u2
checkpoint = add_points(P = u1, Q = u2, p=p)

In [26]:
assert checkpoint[0] ==r

AssertionError: 